In [27]:
import dask.distributed
import pystac_client
import stackstac
import dask.array as da
import dask.dataframe as dd
import geopandas as gpd
from copy import deepcopy
import pandas as pd
from shapely.geometry import shape

#Load STAC items into xarray Datasets. Process locally or distribute data loading and computation with Dask.
from odc.stac import configure_rio, load
from IPython.display import HTML, display


In [13]:
client = dask.distributed.Client()
display(client)

2022-10-18 15:48:42,693 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jys1tlet', purging
2022-10-18 15:48:42,693 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-oq7dar2p', purging
2022-10-18 15:48:42,693 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9dnd8x1g', purging
2022-10-18 15:48:42,693 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ekin39w8', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 24.77 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37689,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 24.77 GiB
Comm: tcp://127.0.0.1:44729,Total threads: 3
Dashboard: http://127.0.0.1:32769/status,Memory: 6.19 GiB
Nanny: tcp://127.0.0.1:34395,


In [14]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

In [15]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"# read in AOI as a GeoDataFrame
filename = "petermann.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)



With the pystac_client module’s Client class, Open the STAC API. 

In [46]:
%%time

bbox =aoi.unary_union.bounds
items = (
    pystac_client.Client.open("https://earth-search.aws.element84.com/v0")
    .search(
        bbox=bbox,
        collections=["sentinel-s2-l2a-cogs"],
        query={"eo:cloud_cover":{"lt":1},"sentinel:valid_cloud_cover": {"eq": True}},
        datetime = "2020-01-01/2020-12-30"
    )
)

items.matched()

CPU times: user 101 ms, sys: 8.02 ms, total: 109 ms
Wall time: 1.35 s


480

In [17]:
# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i['geometry'] = shape(_i['geometry'])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ['properties.datetime', 'properties.created', 'properties.updated']:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index('properties.datetime', inplace=True)
    return gdf

In [47]:
# convert found items to a GeoDataFrame
lst_items=list(items.items_as_dicts())
items_gdf = items_to_geodataframe(lst_items)

In [49]:
%time  
for col in ddf.columns:
    if 'href' in col:
        print(col)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs
assets.overview.href
assets.thumbnail.href
assets.metadata.href
assets.B11.href
assets.B01.href
assets.B12.href
assets.B02.href
assets.B03.href
assets.B04.href
assets.AOT.href
assets.B05.href
assets.B06.href
assets.B07.href
assets.B08.href
assets.B8A.href
assets.B09.href
assets.WVP.href
assets.visual.href
assets.SCL.href
assets.info.href


In [62]:
import requests

def get_url_status(url):  # checks status for each url in list urls
    try:
        r = requests.head(url)
        if r == 404:
            print(url + "\tStatus: " + str(r.status_code))
    except Exception as e:
        print(url + "\tNA FAILED TO CONNECT\t" + str(e))
    

In [81]:
df= items_gdf['assets.SCL.href'].to_frame()
ddf = dd.from_pandas(df, npartitions=10)

def checkurl(df):
    return df.apply(get_url_status)

In [87]:
p= ddf.apply(checkurl, axis=1,meta={'assets.SCL.href': 'object'})
p.compute()

,assets.SCL.href
properties.datetime,
2020-05-16 19:30:31+00:00,None
2020-05-19 18:00:44+00:00,None
2020-05-19 18:00:44+00:00,None
2020-05-19 20:30:13+00:00,None
2020-05-19 20:30:14+00:00,None
...,...
2020-08-18 19:10:49+00:00,None
2020-08-18 19:10:49+00:00,None
2020-08-19 18:40:39+00:00,None
